In [12]:
import numpy as np

import os
import warnings
import pandas as pd
from PIL import Image
from glob import glob
from tqdm.notebook import tqdm
import warnings
import math
import torchvision
import cv2
import matplotlib.pyplot as plt
# 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore")
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [13]:
file_list=glob('../../data/video_data/**/**/*.mp4')
image_file_list=[f.replace('.mp4', '') for f in file_list]
image_file_list=[f.replace('/video_data/', '/image_data_pd/') for f in image_file_list]
for i in tqdm(range(len(file_list))):
    try:
        count=0
        vidcap = torchvision.io.read_video(file_list[i])
        fps=int(vidcap[2]['video_fps'])
        video=np.array(vidcap[0],dtype=np.uint8)
        video_crop=np.zeros((len(video)-1,video.shape[1],video.shape[2],3))
        for j in range(len(video_crop)):
            video_crop[j]=video[j+1]-video[j]
        video_crop=video_crop.sum(axis=0)
        video_crop=video_crop.sum(axis=2)
        video_crop=((video_crop/video_crop.max())*255).astype(np.uint8)
        y1=np.where(video_crop>100)[0].min()
        y2=np.where(video_crop>100)[0].max()
        x1=np.where(video_crop>100)[1].min()
        x2=np.where(video_crop>100)[1].max()
        createDirectory(image_file_list[i])
        for k in range(0,len(video),fps//5):
            img=Image.fromarray(video[k,y1:y2,x1:x2])
            im_new = expand2square(img, (0, 0, 0))
            im_new.resize((256,256)).save(image_file_list[i]+"/%06d.jpg" % count)
            count += 1
    except:
        print(file_list[i])

  0%|          | 0/3823 [00:00<?, ?it/s]